# Chapter 11: Algebraic datatypes

## Multiple choice

In [1]:
data Weekday =
    Monday
    | Tuesday
    | Wednesday
    | Thursday
    | Friday

`Weekday` is a type with five data constructors

In [2]:
f Friday = "Miller Time"

:t f
-- f :: Weekday -> String

f :: Weekday -> [Char]

Types defined with the `data` keyword must begin with a capital letter.

The function `g xs = xs !! (length xs - 1)` delivers the final element of `xs`.

## Ciphers

---

In [3]:
import Data.Char

In [4]:
punctuations = " "
base = ord 'a'
end = ord 'z'
size = end - base + 1

In [5]:
import Data.Maybe

data LetterOrPunctuation = Letter Char | Punctuation Char deriving Show

unLetter :: LetterOrPunctuation -> Char
unLetter (Letter x) = x
unLetter (Punctuation x) = x

getLetter :: Char -> Maybe LetterOrPunctuation
getLetter x
    | x `elem` punctuations = (Just . Punctuation) x
    | l < base = Nothing
    | l > end = Nothing
    | otherwise = (Just . Letter) x
    where l = ord x

getLetterFromInt :: Int -> Maybe LetterOrPunctuation
getLetterFromInt = getLetter . chr

unsafeGetLetter :: Char -> LetterOrPunctuation
unsafeGetLetter = fromJust . getLetter

unsafeGetLetterFromInt :: Int -> LetterOrPunctuation
unsafeGetLetterFromInt = fromJust . getLetterFromInt

caesarLetter :: Int -> LetterOrPunctuation -> LetterOrPunctuation
caesarLetter _ (Punctuation x) = Punctuation x
caesarLetter shift (Letter x) = (unsafeGetLetterFromInt . f . ord) x where
    f i = (i - base + shift) `mod` size + base
    
caesarLetter 3 (Letter 'a')

Letter 'd'

In [6]:
getLetters :: String -> Maybe [LetterOrPunctuation]
getLetters = traverse getLetter

unsafeGetLetters :: String -> [LetterOrPunctuation]
unsafeGetLetters = fromJust . getLetters

unLetters :: [LetterOrPunctuation] -> String
unLetters = fmap unLetter

caesar :: Int -> [LetterOrPunctuation] -> [LetterOrPunctuation]
caesar shift = fmap (caesarLetter shift)

unLetters <$> caesar 3 <$> getLetters "qwer"
unLetters <$> caesar 3 <$> getLetters "qw er"

Just "tzhu"

Just "tz hu"

In [7]:
uncaesar :: Int -> [LetterOrPunctuation] -> [LetterOrPunctuation]
uncaesar shift = caesar (-shift)

unLetters . uncaesar 3 <$> caesar 3 <$> getLetters "qwer"
unLetters . uncaesar 3 <$> caesar 3 <$> getLetters "qw er"

Just "qwer"

Just "qw er"

---

In [8]:
zipIgnoringPunctuation :: [a] -> [LetterOrPunctuation] -> [(Maybe a, LetterOrPunctuation)]
zipIgnoringPunctuation = f where
    f [] _ = []
    f _ [] = []
    f xs (y@(Punctuation _):yt) = (Nothing, y):f xs yt
    f (x:xt) (y:yt) = (Just x, y):f xt yt

In [9]:
zipIgnoringPunctuation [1, 2, 3, 4] <$> getLetters "qwer"
zipIgnoringPunctuation [1, 2, 3, 4] <$> getLetters "qw er"

Just [(Just 1,Letter 'q'),(Just 2,Letter 'w'),(Just 3,Letter 'e'),(Just 4,Letter 'r')]

Just [(Just 1,Letter 'q'),(Just 2,Letter 'w'),(Nothing,Punctuation ' '),(Just 3,Letter 'e'),(Just 4,Letter 'r')]

In [10]:
toShift :: LetterOrPunctuation -> Maybe Int
toShift (Letter x) = Just $ ord x - base
toShift (Punctuation _) = Nothing

In [11]:
import Control.Monad (join)
import Data.Bifunctor (first)

vigenere' :: (Int -> Int) -> [LetterOrPunctuation] -> [LetterOrPunctuation] -> [LetterOrPunctuation]
vigenere' mapShift word s = encoded where
    shifts = cycle . fmap (fmap mapShift . toShift) $ word
    dirtyPairs = zipIgnoringPunctuation shifts s
    pairs = fmap (first join) dirtyPairs
--     f Nothing l = l
--     f (Just s) l = caesarLetter s l
    f s l = fromMaybe l (caesarLetter <$> s <*> pure l)
    encoded = fmap (uncurry f) pairs
    
vigenere :: [LetterOrPunctuation] -> [LetterOrPunctuation] -> [LetterOrPunctuation]
vigenere = vigenere' id

In [12]:
unLetters <$> (vigenere <$> getLetters "ally" <*> getLetters "meet at dawn")

Just "mppr ae oywy"

In [13]:
unvigenere :: [LetterOrPunctuation] -> [LetterOrPunctuation] -> [LetterOrPunctuation]
unvigenere = vigenere' (*(-1))

input = getLetters "meet at dawn"
keyword = getLetters "ally"

code = (vigenere <$> keyword <*>)
uncode = (unvigenere <$> keyword <*>)

coded = code input
uncoded = uncode coded

unLetters <$> uncoded

Just "meet at dawn"

## As-patterns

In [14]:
import Data.Char

In [15]:
isSubsequenceOf :: Eq a => [a] -> [a] -> Bool
isSubsequenceOf [] _ = True
isSubsequenceOf _ [] = False
isSubsequenceOf sub@(c:cs) (c':cs')
    | c == c' = isSubsequenceOf cs cs'
    | otherwise = isSubsequenceOf sub cs'
    
isSubsequenceOf "blah" "blahwoot"
isSubsequenceOf "blah" "wootblah"
isSubsequenceOf "blah" "wboloath"
isSubsequenceOf "blah" "wootbla"
isSubsequenceOf "blah" ""
isSubsequenceOf "" "blahwoot"

True

True

True

False

False

True

In [16]:
capitalizeWords :: String -> [(String, String)]
capitalizeWords = fmap f . words where 
    capitalize [] = []
    capitalize (c:cs) = toUpper c:cs
    tuplify a = (a, a)
    f = fmap capitalize . tuplify

capitalizeWords "hello world"

[("hello","Hello"),("world","World")]

In [17]:
capitalizeWords :: String -> [(String, String)]
capitalizeWords = fmap f . words where
    f [] = ([], [])
    f s@(c:cs) = (s, toUpper c:cs)
    
capitalizeWords "hello world"

[("hello","Hello"),("world","World")]

## Language exercises

In [18]:
capitalizeWord :: String -> String
capitalizeWord [] = []
capitalizeWord (c:cs) = toUpper c:cs

capitalizeWord "Titter"
capitalizeWord "titter"

"Titter"

"Titter"

In [19]:
import Data.List (intercalate)
import Data.List.Split (splitOn)

capitalizeParagraph :: String -> String
capitalizeParagraph = intercalate sep . fmap capitalizeWord . splitOn sep 
    where sep = ". "

capitalizeParagraph "blah. woot ha."

"Blah. Woot ha."

## Phone exercise

In [20]:
import Data.List
import Data.Maybe

newtype PositiveInt = PositiveInt Int deriving Show

makePositiveInt :: Int -> Maybe PositiveInt
makePositiveInt x
    | x > 0 = (Just . PositiveInt) x
    | otherwise = Nothing

unsafeMakePositiveInt :: Int -> PositiveInt
unsafeMakePositiveInt = fromJust . makePositiveInt

data DaPhoneButton = 
    One
    | Two
    | Three
    | Four
    | Five
    | Six
    | Seven
    | Eight
    | Nine
    | Star
    | Zero
    | Hash
    deriving Show

data DaPhoneRawInput = DaPhoneRawInput DaPhoneButton PositiveInt deriving Show

data DaPhoneControl = ToUpper deriving (Show, Eq)

data DaPhoneInput = LetterInput Char | ControlInput DaPhoneControl deriving (Show, Eq)

data DaPhoneChar = 
    CharSimple Char
    | CharComplex DaPhoneControl Char
    deriving Show

sets :: DaPhoneButton -> [DaPhoneInput]
sets One = fmap LetterInput "1"
sets Two = fmap LetterInput "abc2"
sets Three = fmap LetterInput "def3"
sets Four = fmap LetterInput "ghi4"
sets Five = fmap LetterInput "jkl5"
sets Six = fmap LetterInput "mno6"
sets Seven = fmap LetterInput "pqrs7"
sets Eight = fmap LetterInput "tuv8"
sets Nine = fmap LetterInput "wxyz9"
sets Star = ControlInput ToUpper : fmap LetterInput "*"
sets Zero = fmap LetterInput " +_0"
sets Hash = fmap LetterInput ".,#"

cycles :: DaPhoneButton -> [DaPhoneInput]
cycles = cycle . sets

input2raw :: DaPhoneInput -> DaPhoneRawInput
input2raw i
    | Just n <- i `elemIndex` sets One = DaPhoneRawInput One (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Two = DaPhoneRawInput Two (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Three = DaPhoneRawInput Three (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Four = DaPhoneRawInput Four (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Five = DaPhoneRawInput Five (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Six = DaPhoneRawInput Six (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Seven = DaPhoneRawInput Seven (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Eight = DaPhoneRawInput Eight (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Nine = DaPhoneRawInput Nine (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Star = DaPhoneRawInput Star (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Zero = DaPhoneRawInput Zero (PositiveInt (n + 1))
    | Just n <- i `elemIndex` sets Hash = DaPhoneRawInput Hash (PositiveInt (n + 1))

preprocess :: DaPhoneRawInput -> DaPhoneInput
preprocess (DaPhoneRawInput button (PositiveInt n)) = cycles button !! (n - 1)

process' :: [DaPhoneInput] -> String
process' [] = []
process' (ControlInput ToUpper:[]) = undefined
process' (ControlInput ToUpper:ControlInput ToUpper:_) = error "impossible"
process' (ControlInput ToUpper:LetterInput c:is) = toUpper c : process' is
process' (LetterInput c:is) = c : process' is

process :: [DaPhoneRawInput] -> String
process = process' . fmap preprocess

convertChar :: Char -> DaPhoneChar
convertChar c
    | isUpper c = CharComplex ToUpper (toLower c)
    | otherwise = CharSimple c
    
char2input :: DaPhoneChar -> [DaPhoneInput]
char2input (CharSimple c) = [LetterInput c]
char2input (CharComplex control c) = [ControlInput control, LetterInput c]

convertString :: String -> [DaPhoneChar]
convertString = fmap convertChar

reverseProcess :: String -> [DaPhoneRawInput]
reverseProcess = fmap input2raw . (>>= char2input) . convertString

Line 77: Use list literal pattern
Found:
(ControlInput ToUpper : [])
Why not:
[ControlInput ToUpper]

In [21]:
convertString "Wanna"

[CharComplex ToUpper 'w',CharSimple 'a',CharSimple 'n',CharSimple 'n',CharSimple 'a']

In [22]:
reverseProcess "Wanna"

[DaPhoneRawInput Star (PositiveInt 1),DaPhoneRawInput Nine (PositiveInt 1),DaPhoneRawInput Two (PositiveInt 1),DaPhoneRawInput Six (PositiveInt 2),DaPhoneRawInput Six (PositiveInt 2),DaPhoneRawInput Two (PositiveInt 1)]

In [23]:
convo :: [String]
convo = 
    [
        "Wanna play 20 questions",
        "Ya",
        "U 1st haha",
        "Lol ok. Have u ever tasted alcohol lol",
        "Lol ya",
        "Wow ur cool haha. Ur turn",
        "Ok. Do u think I am pretty Lol",
        "Lol ya",
        "Haha thanks just making sure rofl ur turn"
    ]

In [24]:
fmap convertString convo

[[CharComplex ToUpper 'w',CharSimple 'a',CharSimple 'n',CharSimple 'n',CharSimple 'a',CharSimple ' ',CharSimple 'p',CharSimple 'l',CharSimple 'a',CharSimple 'y',CharSimple ' ',CharSimple '2',CharSimple '0',CharSimple ' ',CharSimple 'q',CharSimple 'u',CharSimple 'e',CharSimple 's',CharSimple 't',CharSimple 'i',CharSimple 'o',CharSimple 'n',CharSimple 's'],[CharComplex ToUpper 'y',CharSimple 'a'],[CharComplex ToUpper 'u',CharSimple ' ',CharSimple '1',CharSimple 's',CharSimple 't',CharSimple ' ',CharSimple 'h',CharSimple 'a',CharSimple 'h',CharSimple 'a'],[CharComplex ToUpper 'l',CharSimple 'o',CharSimple 'l',CharSimple ' ',CharSimple 'o',CharSimple 'k',CharSimple '.',CharSimple ' ',CharComplex ToUpper 'h',CharSimple 'a',CharSimple 'v',CharSimple 'e',CharSimple ' ',CharSimple 'u',CharSimple ' ',CharSimple 'e',CharSimple 'v',CharSimple 'e',CharSimple 'r',CharSimple ' ',CharSimple 't',CharSimple 'a',CharSimple 's',CharSimple 't',CharSimple 'e',CharSimple 'd',CharSimple ' ',CharSimple 'a',Ch

In [25]:
keypresses :: DaPhoneRawInput -> PositiveInt
keypresses (DaPhoneRawInput _ n) = n

unPositiveInt :: PositiveInt -> Int
unPositiveInt (PositiveInt n) = n

fingerTaps :: Char -> Int
fingerTaps = sum . fmap (unPositiveInt . keypresses . input2raw) . char2input . convertChar

convertString "Wanna"
fmap fingerTaps "Wanna"
fmap fingerTaps "wanna"

[CharComplex ToUpper 'w',CharSimple 'a',CharSimple 'n',CharSimple 'n',CharSimple 'a']

[2,1,2,2,1]

[1,1,2,2,1]

In [26]:
import Data.Function (on)

mostPopular :: Ord a => [a] -> a
mostPopular = head . maximumBy (compare `on` length) . group . sort

mostPopularLetter :: String -> Char
mostPopularLetter = mostPopular . filter (/= ' ')

l = mostPopularLetter "asdfasdfzzzZZZZZ                "
l
mostPopularLetter ""

'Z'

: 

In [27]:
fingerTaps l

5

In [28]:
mostPopularLetterOverall :: [String] -> Char
mostPopularLetterOverall = mostPopularLetter . join

mostPopularLetterOverall convo

'a'

In [29]:
mostPopularWord :: String -> String
mostPopularWord = mostPopular . words

mostPopularWord "hello world hello"

"hello"

In [30]:
mostPopularWordOverall :: [String] -> String
mostPopularWordOverall = mostPopularWord . unwords

mostPopularWordOverall convo

"Lol"

In [31]:
mostPopularWordOverallIgnoringCase = mostPopularWord . fmap toLower . unwords $ convo
mostPopularWordOverallIgnoringCase

"lol"

## Hutton's Razor

In [32]:
data Expr
    = Lit Integer
    | Add Expr Expr
    
eval :: Expr -> Integer
eval (Lit x) = x
eval (Add e1 e2) = ((+) `on` eval) e1 e2

eval (Add (Lit 1) (Lit 9001))
9002

9002

9002

In [33]:
instance Show Expr where
    show (Lit x) = show x
    show (Add e1 e2) = show' e1 ++ " + " ++ show' e2 where
        show' e@(Lit _) = show e
        show' e = "(" ++ show e ++ ")"

Add (Lit 1) (Add (Lit 2) (Lit 55))

a1 = Add (Lit 9001) (Lit 1)
a2 = Add a1 (Lit 20001)
a3 = Add (Lit 1) a2
a3

1 + (2 + 55)

1 + ((9001 + 1) + 20001)